In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage

import pandas as pd
from io import StringIO

import os
import json
from collections import namedtuple

ChatGPTAnswer = namedtuple("ChatGPTAnswer", ["thoughts", "csv_output"])

In [2]:
SYS_PROMPT = (
    "You are a professional network graph maker who extracts "
    "ontology from a given text on the topic of Euclidian geometry. "
    "You are provided with a context chunk (delimited by +++++). "
    "Your MUST extract key geometric entities from the context that are "
    "connected by a 'is-a', 'is a kind of' or 'is a type of' relationship.\n"
    "Thought 1: While traversing through each sentence, think about the key geometric entities mentioned in it.\n"
    "   Entities are typically geometrical objects such as points, lines, squares, circles, elliptic curves, or their constituents.\n"
    "   Also think of the terms or concepts that might be implicit in the text.\n"
    "   Terms must be as atomistic as possible, usually in singular rather than plural."
    "Thought 2: Think about the 'is-a' hierarchy between the entities you extracted.\n"
    "   Make sure you are not confusing 'is-a' with 'has-a'. For instance, a center is not a circle, but rather a circle has a center.\n"
    "   Which of the entities fit nicely into the '<X> is a kind of <Y>' template? Are there hyponym-hypernym pairs among them?\n"
    "   X might be a subtype or subvariety of Y, like inheritance in object-oriented programming languages.\n\n"
)

CONTEXT_USER_PROMPT = "Context:\n\n+++++\n{context}\n+++++\n\nYour thoughs:\n"
FORMATTING_USER_PROMPT = (
    "Now format your final response to make it suitable for parsing in the simple CSV format. "
    "For each pair of entities such that X is a kind of Y, put them on a single line, separated by a comma:\n"
    "X,Y\n"
    "You will be PENALIZED for every deviation from the CSV format.\n"
    "You will be PENALIZED for false or lousy relationships, so ENSURE that\n"
    "    - every X and Y are indeed a legitimate geometrical entity\n"
    "    - every X is indeed a kind of Y, meaning that X derives all properties of Y.\n"
    "Your output in the CSV format:\n"
)

def get_x_is_a_kind_of_y_from_chunk(context_chunk: str) -> ChatGPTAnswer:       
    chat = ChatOpenAI()
    messages = [
        SystemMessage(content=SYS_PROMPT),
        HumanMessage(content=CONTEXT_USER_PROMPT.format(context=context_chunk)),
    ]
    thoughts = chat.invoke(messages)
    csv_output = chat.invoke(messages + [thoughts, HumanMessage(content=FORMATTING_USER_PROMPT)])
    return ChatGPTAnswer(thoughts.content, csv_output.content)

In [54]:
XIsAYPairs = list[tuple[str, str]]

def parse_chatgpt_csv_answer(csv_answer: str) -> XIsAYPairs:
    df = pd.read_csv(StringIO(csv_answer), names=["x_is_a_kind", "of_y"])
    return [tuple(row) for row in df.itertuples(index=False, name=None)]

def calculate_metrics(ground_truth: XIsAYPairs, llm_answers: XIsAYPairs) -> dict:
    ground_truth = [(t.lower().strip(), T.lower().strip()) for t, T in ground_truth]
    llm_answers = [(l.lower().strip(), L.lower().strip()) for l, L in llm_answers]
    
    # Validation step (skipped)
    # Duplicates removed when set is constructed.
    ground_truth = set(ground_truth)
    llm_answers = set(llm_answers)
    
    exact_matches = set()
    for gt_pair in ground_truth:
        if gt_pair in llm_answers:
            exact_matches.add(gt_pair)
    
    wrong_answers = llm_answers.difference(exact_matches)
    unpaired_ground_truth = ground_truth.difference(exact_matches)
    
    if len(ground_truth) == 0:
        recall = 1
    else:
        recall = len(exact_matches) / len(ground_truth)
    
    if len(llm_answers) == 0:
        trash_rate = 0
    else:
        trash_rate = len(wrong_answers) / len(llm_answers)
    
    metrics = {
        "exact_matches" : exact_matches,
        "wrong_answers" : wrong_answers,
        "unpaired_ground_truth" : unpaired_ground_truth,
        "recall" : recall,
        "trash_rate" : trash_rate,
    }
    return metrics

In [4]:
with open("annotations.json") as file:
    annotated_chunks = json.load(file)["annotated_chunks"]

In [5]:
first = annotated_chunks[0]
chunk, xisypairs = first["chunk"], first["x_is_a_y"]
xisypairs

[['polygon', 'plane figure'],
 ['closed polygonal chain', 'polygon'],
 ['vertex', 'point'],
 ['edge', 'segment'],
 ['side', 'segment'],
 ['n-gon', 'polygon'],
 ['triangle', 'n-gon'],
 ['3-gon', 'n-gon'],
 ['triangle', 'n-gon'],
 ['simple polygon', 'boundary'],
 ['simple polygon', 'polygon'],
 ['solid polygon', 'region'],
 ['solid polygon', 'polygon'],
 ['skew polygon', 'polygon'],
 ['self-intersecting polygon', 'polygon'],
 ['star polygon', 'polygon'],
 ['polygon', 'polytope']]

In [8]:
res = get_x_is_a_kind_of_y_from_chunk(chunk)
res

ChatGPTAnswer(thoughts="Key geometric entities extracted from the text:\n1. Polygon\n2. Plane figure\n3. Line segment\n4. Closed polygonal chain\n5. Edge/side\n6. Vertex/corner\n7. n-gon\n8. Triangle\n9. Simple polygon\n10. Intersection\n11. Shared endpoint\n12. Solid polygon\n13. Body\n14. Polygonal region/polygonal area\n15. Star polygon\n16. Self-intersecting polygon\n17. Skew polygon\n18. Polytope\n\nHierarchy of geometric entities:\n- A polygon is a plane figure.\n- A polygon is made up of line segments connected to form a closed polygonal chain.\n- A closed polygonal chain consists of edges or sides.\n- The points where two edges meet are the polygon's vertices or corners.\n- An n-gon is a polygon with n sides.\n- A triangle is a type of 3-gon.\n- A simple polygon is a type of polygon which does not intersect itself.\n- A solid polygon is a type of polygon that has a boundary known as a simple polygon.\n- The interior of a solid polygon is its body, also known as a polygonal regi

In [9]:
print(res.csv_output)

Polygon,Plane figure
Polygon,Line segment
Polygon,Closed polygonal chain
Closed polygonal chain,Edge
Closed polygonal chain,Vertex
n-gon,Polygon
Triangle,n-gon
Simple polygon,Polygon
Solid polygon,Polygon
Body,Solid polygon
Star polygon,Self-intersecting polygon
Skew polygon,Closed polygonal chain
Polygon,2-dimensional polytope


In [18]:
llm_answers = parse_chatgpt_csv_answer(res.csv_output)
llm_answers

[('Polygon', 'Plane figure'),
 ('Polygon', 'Line segment'),
 ('Polygon', 'Closed polygonal chain'),
 ('Closed polygonal chain', 'Edge'),
 ('Closed polygonal chain', 'Vertex'),
 ('n-gon', 'Polygon'),
 ('Triangle', 'n-gon'),
 ('Simple polygon', 'Polygon'),
 ('Solid polygon', 'Polygon'),
 ('Body', 'Solid polygon'),
 ('Star polygon', 'Self-intersecting polygon'),
 ('Skew polygon', 'Closed polygonal chain'),
 ('Polygon', '2-dimensional polytope')]

In [21]:
calculate_metrics(xisypairs, llm_answers)

{'exact_matches': {('n-gon', 'polygon'),
  ('polygon', 'plane figure'),
  ('simple polygon', 'polygon'),
  ('solid polygon', 'polygon'),
  ('triangle', 'n-gon')},
 'wrong_answers': {('body', 'solid polygon'),
  ('closed polygonal chain', 'edge'),
  ('closed polygonal chain', 'vertex'),
  ('polygon', '2-dimensional polytope'),
  ('polygon', 'closed polygonal chain'),
  ('polygon', 'line segment'),
  ('skew polygon', 'closed polygonal chain'),
  ('star polygon', 'self-intersecting polygon')},
 'unpaired_ground_truth': {('3-gon', 'n-gon'),
  ('closed polygonal chain', 'polygon'),
  ('edge', 'segment'),
  ('polygon', 'polytope'),
  ('self-intersecting polygon', 'polygon'),
  ('side', 'segment'),
  ('simple polygon', 'boundary'),
  ('skew polygon', 'polygon'),
  ('solid polygon', 'region'),
  ('star polygon', 'polygon'),
  ('vertex', 'point')},
 'recall': 0.3125,
 'trash_rate': 0.3846153846153846}

In [22]:
chatgpt_answers = []
for anck in annotated_chunks:
    answer = get_x_is_a_kind_of_y_from_chunk(anck["chunk"])
    chatgpt_answers.append(answer)

In [40]:
import pickle

with open("20240227_chatgpt_answers.pickle", "wb") as file:
    pickle.dump(chatgpt_answers, file)

In [45]:
all_metrics = []
for i, answer in enumerate(chatgpt_answers):
    try:
        llm_answers = parse_chatgpt_csv_answer(answer.csv_output)
    except Exception as e:
        print(answer.csv_output)
        llm_answers = []
        
    ground_truth = annotated_chunks[i]["x_is_a_y"]
    m = calculate_metrics(ground_truth, llm_answers)
    all_metrics.append(m)

Polygon,Geometric shape
Greek adjective,Adjective
πολύς (polús) 'much', 'many',Greek adjective
γωνία (gōnía) 'corner' or 'angle',Geometric property
γόνυ (gónu) 'knee',Body part


In [47]:
[m["recall"] for m in all_metrics]

[0.3125,
 1,
 0.7857142857142857,
 0.25,
 0.3333333333333333,
 0.0,
 0.25,
 0.6666666666666666,
 1,
 0.5833333333333334,
 0.8571428571428571,
 0.0,
 0.0,
 0.5]

In [48]:
[m["trash_rate"] for m in all_metrics]

[0.3125,
 0,
 0.8461538461538461,
 0.1428571428571429,
 0.16666666666666663,
 0.0,
 0.25,
 0.19999999999999996,
 0.0,
 0.6363636363636364,
 0.8571428571428572,
 0.0,
 0.0,
 0.07692307692307687]

In [58]:
for m in all_metrics:
    pprint.pprint(m)
    print("\n\n")

{'exact_matches': {('n-gon', 'polygon'),
                   ('polygon', 'plane figure'),
                   ('simple polygon', 'polygon'),
                   ('skew polygon', 'polygon'),
                   ('solid polygon', 'polygon')},
 'recall': 0.3125,
 'trash_rate': 0.3125,
 'unpaired_ground_truth': {('3-gon', 'n-gon'),
                           ('closed polygonal chain', 'polygon'),
                           ('edge', 'segment'),
                           ('polygon', 'polytope'),
                           ('self-intersecting polygon', 'polygon'),
                           ('side', 'segment'),
                           ('simple polygon', 'boundary'),
                           ('solid polygon', 'region'),
                           ('star polygon', 'polygon'),
                           ('triangle', 'n-gon'),
                           ('vertex', 'point')},
 'wrong_answers': {('body of a polygon', 'solid polygon'),
                   ('closed polygonal chain', 'edges or sides'